### Imports and Environment Setup

In [ ]:
# Enable inline plotting in Jupyter Notebook
%matplotlib inline

# Import essential libraries for image processing, plotting, and handling data formats
import cv2
import scipy.io
import os
import sys
import json
import math
import time
import random
import pickle
import h5py
import glob
import scipy.spatial
import scipy.ndimage
import numpy as np
from scipy import stats
from random import shuffle
from matplotlib import pyplot as plt
from skimage.transform import downscale_local_mean
from sklearn.model_selection import train_test_split

# Set up Caffe environment (update 'caffe_root' with your installation location)
caffe_root = os.path.expanduser('~/deeplab-public-ver2/')
sys.path.insert(0, os.path.join(caffe_root, 'python'))
sys.path.insert(0, os.path.join(caffe_root, 'python/caffe/proto'))

# Import Caffe modules for model operations
import caffe
import caffe_pb2

### Constants Setup

In [ ]:
# Define paths for the model, dataset, and weights
model_name = 'CrowdNet'
model_path = os.path.expanduser(os.path.join('models', model_name))  # Model definition path
data_path = os.path.expanduser(os.path.join('~/data', model_name))   # Dataset location
weights_path = os.path.expanduser(os.path.join('~/models', model_name))  # Pre-trained weights path

# Dataset paths (can be expanded if multiple datasets are used)
dataset_paths = ['dataset/UCF_CC_50']

# Define slicing and patch dimensions for image processing
slice_w = 256  # Width of each slice
slice_h = 256  # Height of each slice

patch_w = 225  # Width of each patch
patch_h = 225  # Height of each patch

# Define the network output size for density maps
net_density_h = 28  # Height of the density map output
net_density_w = 28  # Width of the density map output

# GPU settings
HAS_GPU = True  # Set to False if you want to use CPU
GPU_ID = 0  # GPU ID to be used

### VGG Mean Initialization

In [ ]:
# Initialize the mean values used for VGG input normalization
VGG_ILSVRC_16_layers_mean = np.zeros((3, patch_h, patch_w), dtype='f4')

# Assign RGB mean values (from ImageNet dataset)
VGG_ILSVRC_16_layers_mean[0, :, :] = 103.939  # Mean for the blue channel
VGG_ILSVRC_16_layers_mean[1, :, :] = 116.779  # Mean for the green channel
VGG_ILSVRC_16_layers_mean[2, :, :] = 123.68   # Mean for the red channel

### Load Ground Truth from JSON

In [ ]:
# Load ground truth from JSON file and populate a matrix (gt_matrix)
def parse_gt_from_json(gt_json_file, matrix_shape):
    gt_matrix = np.zeros(matrix_shape, dtype='uint8')  # Initialize the ground truth matrix
    with open(gt_json_file, 'r') as json_file:
        for idx, point in enumerate(json.load(json_file)):
            try:
                # Place 1 at the position defined by 'x' and 'y' in the ground truth
                gt_matrix[int(math.floor(point['y'])), int(math.floor(point['x']))] = 1
            except IndexError:
                # Handle any indexing errors and print the problematic data point
                print(gt_json_file, point['y'], point['x'], sys.exc_info())
    return gt_matrix

### Gaussian Filter for Density Map Generation

In [ ]:
# Apply Gaussian filter to generate density maps from ground truth points
def gaussian_smooth_density(gt_list):
    density_maps = []  # List to store generated density maps

    # Iterate over each ground truth (gt) map
    for gt_matrix in gt_list:
        print(gt_matrix.shape)
        density_map = np.zeros(gt_matrix.shape, dtype=np.float32)  # Initialize density map
        non_zero_count = np.count_nonzero(gt_matrix)  # Count the number of non-zero points (people)

        # If no points are present in the ground truth, return the empty density map
        if non_zero_count == 0:
            return density_map

        # Get the (x, y) coordinates of the points in the ground truth
        points = np.array(list(zip(np.nonzero(gt_matrix)[1], np.nonzero(gt_matrix)[0])))

        # Build a KDTree for efficient nearest neighbor search
        kd_tree = scipy.spatial.KDTree(points.copy(), leafsize=2048)

        # Query the KDTree to find the distances to the nearest neighbors
        distances, locations = kd_tree.query(points, k=2, eps=10.)

        # Generate the density map using Gaussian filters
        for idx, point in enumerate(points):
            pt_2d = np.zeros(gt_matrix.shape, dtype=np.float32)
            pt_2d[point[1], point[0]] = 1.  # Place a 1 at the location of the person

            # Calculate the sigma for the Gaussian filter
            if non_zero_count > 1:
                sigma = distances[idx][1]  # Distance to the nearest neighbor
            else:
                sigma = np.average(np.array(gt_matrix.shape)) / 4.  # Default sigma for a single point

            # Apply Gaussian filter to the point
            density_map += scipy.ndimage.filters.gaussian_filter(pt_2d, sigma, mode='constant')

        density_maps.append(density_map)  # Append the generated density map

    return density_maps

### Load Images, Ground Truths, and Density Maps

In [ ]:
# Load images, ground truths, and density maps from the specified path
def fetch_images_and_gts(path):
    images = []  # Store loaded images
    gts = []     # Store ground truth matrices
    densities = []  # Store density maps

    # Iterate over all ground truth files in the specified path
    for gt_file in glob.glob(os.path.join(path, '*.json')):
        print(gt_file)

        # Load the corresponding image (either PNG or JPG)
        if os.path.isfile(gt_file.replace('.json','.png')):
            img = cv2.imread(gt_file.replace('.json','.png'))
        else:
            img = cv2.imread(gt_file.replace('.json','.jpg'))
        images.append(img)
        
        # Load the ground truth using the previously defined function
        gt = parse_gt_from_json(gt_file, img.shape[:-1])
        gts.append(gt)
        
        # Load or generate the density map
        density_file = gt_file.replace('.json','.h5')
        if os.path.isfile(density_file):
            # Load density map from file if it exists
            with h5py.File(density_file, 'r') as hf:
                density = np.array(hf.get('density'))
        else:
            # Generate the density map if not found and save it
            density = gaussian_smooth_density([gt])[0]
            with h5py.File(density_file, 'w') as hf:
                hf['density'] = density
        densities.append(density)

    # Output the number of images and ground truth files loaded
    print(path, len(images), 'loaded')
    return (images, gts, densities)

### Density Resizing and Multiscale Pyramidal Processing

In [ ]:
# Resize the density map, scaling by factors scale_x and scale_y
def resize_density_map(density_map, scale_x, scale_y):
    return cv2.resize(density_map, None, fx=scale_x, fy=scale_y, interpolation=cv2.INTER_CUBIC) / (scale_x * scale_y)

# Apply multiscale pyramidal transformations to images and corresponding ground truths
def apply_multiscale_transform(image_list, gt_list, scale_start=0.5, scale_end=1.3, scale_step=0.1):
    scale_factors = np.arange(scale_start, scale_end, scale_step)  # Range of scale factors
    multiscale_images = []  # To store scaled images
    multiscale_gts = []     # To store scaled ground truths
    
    # For each image, apply scaling with the specified factors
    for idx, img in enumerate(image_list):
        for scale in scale_factors:
            multiscale_images.append(cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC))
            multiscale_gts.append(resize_density_map(gt_list[idx], fx=scale, fy=scale))
    
    return (multiscale_images, multiscale_gts)

### Image and Density Adaptation for Slicing

In [ ]:
# Adapt images and density maps to match the desired slicing dimensions
def resize_images_and_densities(image_list, gt_list, slice_width=slice_w, slice_height=slice_h):
    resized_images = []  # Store the resized images
    resized_gts = []     # Store the resized ground truths
    
    # Adjust image and ground truth sizes for consistent slicing
    for idx, image in enumerate(image_list):
        img_height, img_width, _ = image.shape
        num_slices_height = int(round(img_height / slice_height))
        num_slices_width = int(round(img_width / slice_width))
        
        new_img_height = float(num_slices_height * slice_height)
        new_img_width = float(num_slices_width * slice_width)
        scale_x = new_img_width / img_width
        scale_y = new_img_height / img_height
        
        # Resize the image and ensure it's aligned with the slicing grid
        resized_images.append(cv2.resize(image, None, fx=scale_x, fy=scale_y, interpolation=cv2.INTER_CUBIC))
        assert resized_images[-1].shape[0] % slice_height == 0 and resized_images[-1].shape[1] % slice_width == 0
        
        # If ground truths are provided, resize them as well
        if gt_list is not None:
            resized_gts.append(resize_density_map(gt_list[idx], fx=scale_x, fy=scale_y))
    
    return (resized_images, resized_gts)

### Generate Slices from Images and Ground Truths

In [ ]:
# Generate slices from images and ground truths
def create_image_slices(image_list, gt_list, slice_width=slice_w, slice_height=slice_h, slice_offset=None):
    if slice_offset is None:
        slice_offset = slice_width  # Default offset is the width of a slice
    
    image_slices = []  # To store the sliced images
    gt_slices = []     # To store the sliced ground truths
    
    # For each image, generate slices based on the specified slicing dimensions and offset
    for idx, image in enumerate(image_list):
        img_height, img_width, _ = image.shape
        y_start, y_end = 0, slice_height
        
        # Loop over height
        while y_end <= img_height:
            x_start, x_end = 0, slice_width
            
            # Loop over width
            while x_end <= img_width:
                # Slice the image and append it
                image_slices.append(image[y_start:y_end, x_start:x_end])
                assert image_slices[-1].shape[:-1] == (slice_height, slice_width)
                
                # Slice the ground truth if provided
                if gt_list is not None:
                    gt_slices.append(gt_list[idx][y_start:y_end, x_start:x_end])
                    assert gt_slices[-1].shape == (slice_height, slice_width)
                
                # Move to the next slice in width
                x_start += slice_offset
                x_end += slice_offset
            
            # Move to the next slice in height
            y_start += slice_offset
            y_end += slice_offset
    
    return (image_slices, gt_slices)

### Data Augmentation (Cropping)

In [ ]:
# Perform data augmentation by cropping patches from images and ground truths
def crop_image_patches(image_list, gt_list):
    cropped_images = []  # Store cropped images
    cropped_gts = []     # Store cropped ground truths
    
    # For each image, crop patches from different regions
    for idx, image in enumerate(image_list):
        img_height, img_width, _ = image.shape
        gt_matrix = gt_list[idx]
        
        # Define positions to crop: top-left, top-right, bottom-left, bottom-right, and center patches
        patch_positions = [
            (0, 0),  # Top-left
            (0, img_width - patch_w),  # Top-right
            (img_height - patch_h, 0),  # Bottom-left
            (img_height - patch_h, img_width - patch_w),  # Bottom-right
            (int((img_height - patch_h) / 2), int((img_width - patch_w) / 2))  # Center
        ]
        
        # Crop the image and ground truth at each position
        for y_start, x_start in patch_positions:
            y_end, x_end = y_start + patch_h, x_start + patch_w
            cropped_images.append(image[y_start:y_end, x_start:x_end])
            cropped_gts.append(gt_matrix[y_start:y_end, x_start:x_end])
    
    return (cropped_images, cropped_gts)

### Data Augmentation (Flipping)

In [ ]:
# Perform data augmentation by flipping images and ground truths
def flip_image_slices(image_list, gt_list):
    flipped_images = []  # Store flipped images
    flipped_gts = []     # Store flipped ground truths
    
    # For each image, add the original and the horizontally flipped version
    for idx, image in enumerate(image_list):
        gt_matrix = gt_list[idx]
        
        # Append original image and ground truth
        flipped_images.append(image)
        flipped_gts.append(gt_matrix)
        
        # Append horizontally flipped image and ground truth
        flipped_images.append(np.fliplr(image))
        flipped_gts.append(np.fliplr(gt_matrix))
    
    return (flipped_images, flipped_gts)

### Shuffling Slices

In [ ]:
# Shuffle images and ground truths to randomize the order
def shuffle_image_slices(image_list, gt_list):
    shuffled_images = []  # Store shuffled images
    shuffled_gts = []     # Store shuffled ground truths
    
    # Create a list of shuffled indices based on the number of images
    shuffled_indices = list(range(len(image_list)))
    shuffle(shuffled_indices)  # Shuffle the indices
    
    # Reorder images and ground truths according to the shuffled indices
    for idx in shuffled_indices:
        shuffled_images.append(image_list[idx])
        shuffled_gts.append(gt_list[idx])
    
    return (shuffled_images, shuffled_gts)

### Sample Distribution

In [ ]:
# Filter and select samples based on density distribution (positive and negative samples)
def select_samples_by_density(image_list, gt_list):
    selected_images = []  # Store selected images
    selected_gts = []     # Store selected ground truths
    gt_counts = list(map(np.sum, gt_list))  # Compute sum of ground truths (i.e., crowd count)
    max_count = max(gt_counts)  # Maximum count in the ground truths
    
    # Select positive samples (crowd density > 0)
    for idx, image in enumerate(image_list):
        if gt_counts[idx] >= 1 and random.random() < (gt_counts[idx]**2) / (max_count**2):
            selected_images.append(image)
            selected_gts.append(gt_list[idx])
    
    # Select negative samples (crowd density = 0)
    negative_count = sum(count < 1 for count in gt_counts)
    target_negative_count = len(selected_gts) // 6  # Target for negative samples
    negative_keep_prob = min(1., float(target_negative_count) / float(negative_count))
    
    # Append negative samples based on the probability
    for idx, image in enumerate(image_list):
        if gt_counts[idx] < 1 and random.random() < negative_keep_prob:
            selected_images.append(image)
            selected_gts.append(gt_list[idx])
    
    return (selected_images, selected_gts)

### Positive Image and Ground Truth Loading

In [ ]:
# Initialize lists for storing images and their corresponding density maps
image_data = []  # List to store images
density_data = []  # List to store density maps

# Loop through each dataset path and load the images, ground truths, and densities
for dataset_path in dataset_paths:
    images, ground_truths, densities = fetch_images_and_gts(dataset_path)  # Load images and density maps for each path
    image_data += images  # Append the loaded images to the list
    density_data += densities  # Append the corresponding density maps to the list

### Split Dataset into Training and Test Sets

In [ ]:
# Split the dataset: 80% for training, 20% for testing
image_data_train, image_data_test, density_data_train, density_data_test = train_test_split(
    image_data, density_data, test_size=0.2  # Reserve 20% of data for testing
)

### Data Augmentation and Preprocessing

In [ ]:
def augment_and_slice_data(images, densities, is_train=True):
    """Applies multiscale pyramidal transformations, generates slices, flips, corrects sample distributions,
    and shuffles the data. If `is_train` is True, it adjusts accordingly."""
    
    # Apply multiscale pyramidal transformations
    print(f'\nMultiscale pyramidal - {"TRAIN" if is_train else "TEST"}:')
    images, densities = apply_multiscale_transform(images, densities)
    print(f'After multiscale pyramidal: {len(images)} images, {len(densities)} density maps')
    
    # Generate patches (slices) from the images and ground truths
    print('\nGenerate slices:')
    images, densities = create_image_slices(images, densities, slice_w=patch_w, slice_h=patch_h, offset=8 if is_train else None)
    print(f'After generating slices: {len(images)} images, {len(densities)} density maps')
    
    # Apply horizontal flip augmentation
    print('\nApply horizontal flip:')
    images, densities = flip_image_slices(images, densities)
    print(f'After flipping: {len(images)} images, {len(densities)} density maps')
    
    # If it's training data, correct the sample distribution
    if is_train:
        print('\nCorrect sample distribution (train):')
        images, densities = select_samples_by_density(images, densities)
        print(f'After sample distribution correction: {len(images)} images, {len(densities)} density maps')
    
    # Shuffle the data to randomize the order
    print('\nShuffle data:')
    images, densities = shuffle_image_slices(images, densities)
    print(f'After shuffling: {len(images)} images, {len(densities)} density maps')
    
    return images, densities

# Apply data augmentation and preprocessing for both train and test sets
image_train, density_train = augment_and_slice_data(image_data_train, density_data_train, is_train=True)
image_test, density_test = augment_and_slice_data(image_data_test, density_data_test, is_train=False)

### Plot Distribution of TRAIN Samples

In [ ]:
# Calculate the sum of density values for each sample in the training set
train_density_sums = [np.sum(density_map) for density_map in density_train]

# Sort the sums for better visualization
train_density_sums.sort()

# Plot the sorted sums
plt.plot(train_density_sums)
plt.title('Distribution of TRAIN Sample Density Sums')
plt.xlabel('Sample Index')
plt.ylabel('Sum of Densities')
plt.show()

### Write Data to HDF5 Files

In [ ]:
# Function to process data and save it in HDF5 format
def process_and_export_hdf5(image_list, density_list, save_path, phase, mean_values):
    # Ensure the target path exists
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    batch_size = 7000  # Set batch size to avoid hitting HDF5 file size limits
    
    # Initialize arrays for processing images and labels in batches
    image_batch = np.zeros((batch_size, 3, patch_h, patch_w), dtype=np.float32)  # Image batch
    density_batch = np.zeros((batch_size, net_density_h, net_density_w), dtype=np.float32)  # Density map batch
    
    # Open a text file to store the names of the generated HDF5 files
    with open(os.path.join(save_path, phase + '.txt'), 'w') as file_log:
        batch_start = 0  # Start index for batch processing
        
        while batch_start < len(image_list):
            # Determine the end index for the current batch
            batch_end = min(batch_start + batch_size, len(image_list))
            
            print(f'{batch_start} - {batch_end - 1} / {len(image_list)}')  # Print progress

            # Create an HDF5 file to store the current batch
            hdf5_filename = os.path.join(save_path, f'{phase}_{batch_start}.h5')
            with h5py.File(hdf5_filename, 'w') as hf:
                # Process the current batch
                for idx, image in enumerate(image_list[batch_start:batch_end]):
                    # Transpose the image and subtract the mean
                    image_batch[idx] = image.copy().transpose(2, 0, 1).astype(np.float32) - mean_values
                    # Resize the corresponding density map and store it
                    density_batch[idx] = resize_density_map(
                        density_list[batch_start + idx],
                        fx=float(net_density_w) / patch_w,
                        fy=float(net_density_h) / patch_h
                    )
                # Write the processed batch to the HDF5 file
                hf['data'] = image_batch[:(batch_end - batch_start)]
                hf['label'] = density_batch[:(batch_end - batch_start)]
            
            # Log the HDF5 file name in the text file
            file_log.write(hdf5_filename + '\n')

            # Move to the next batch
            batch_start += batch_size

# Train set processing
print('TRAIN:')
process_and_export_hdf5(image_train, density_train, data_path, 'train', VGG_ILSVRC_16_layers_mean)

# Test set processing
print('TEST:')
process_and_export_hdf5(image_test, density_test, data_path, 'test', VGG_ILSVRC_16_layers_mean)

### Caffe Model Training

In [ ]:
# Use Caffe's command-line interface to start training the model.
# The specified solver configuration and pre-trained weights are provided.

# Run the Caffe training command using the solver and pre-trained weights.
# This command assumes that the Caffe framework is installed and configured properly.
!caffe train \
    -solver models/CrowdNet/solver.prototxt \
    -weights weights/VGG_ILSVRC_16_layers/VGG_ILSVRC_16_layers.caffemodel

### Image Processing Functions

In [ ]:
# Process a single image: subtract the mean and transpose dimensions
def process_single_image(image, mean_values):
    """
    Process a single image by transposing it from (H, W, C) to (C, H, W) 
    and subtracting the mean values for normalization.
    """
    processed_image = image.copy()  # Create a copy to avoid modifying the original
    processed_image = processed_image.transpose(2, 0, 1).astype(np.float32)  # Transpose (H, W, C) -> (C, H, W)
    processed_image -= mean_values  # Subtract the mean values for normalization
    return processed_image

# Process a batch of images: apply image processing to each image in the batch
def process_image_batch(image_list, mean_values):
    """
    Process a batch of images by applying the image_process function to each image.
    The batch will have the shape (batch_size, channels, height, width).
    """
    # Initialize an empty batch with the appropriate shape (batch_size, channels, height, width)
    batch = np.zeros((len(image_list),) + image_list[0].transpose(2, 0, 1).shape, dtype=np.float32)
    
    # Process each image in the batch
    for idx, image in enumerate(image_list):
        batch[idx] = process_single_image(image, mean_values)
    
    return batch  # Return the processed batch

### Prediction with Caffe Model

In [ ]:
# Perform predictions using the Caffe model on a batch of images
def predict(image_list, mean_values):
    """
    Perform predictions on a list of images using the Caffe model.
    The output is a list of predicted density maps.
    """
    predictions = []  # To store predicted outputs (density maps)

    # Iterate over each image in the input dataset
    for idx, image in enumerate(image_list):
        # Adapt the image to match the network's required input dimensions
        adapted_image, _ = resize_images_and_densities([image], None, slice_w=patch_w, slice_h=patch_h)
        
        # Generate patches (slices) from the adapted image
        image_slices, _ = create_image_slices(adapted_image, None, slice_w=patch_w, slice_h=patch_h, offset=None)
        
        # Define the layer from which to extract the output
        output_layer = 'conv6'  # Use the output from the 'conv6' layer
        batch_size = 10  # Process images in batches of 10
        
        image_predictions = []  # Store predictions for the current image
        batch_start = 0  # Start index for batch processing
        
        # Process the image slices in batches
        while batch_start < len(image_slices):
            # Define the end index for the current batch
            batch_end = min(batch_start + batch_size, len(image_slices))
            
            # Process the batch of images (subtract mean, reshape, etc.)
            batch = process_image_batch(image_slices[batch_start:batch_end], mean_values)
            net.blobs['data'].reshape(batch.shape[0], batch.shape[1], batch.shape[2], batch.shape[3])  # Reshape network input
            net.blobs['data'].data[...] = batch  # Set network input with the processed batch
            
            # Perform forward pass through the network
            net.forward()
            
            # Extract predictions from the specified output layer
            for output in net.blobs[output_layer].data:
                predicted_density = output[0]  # Extract the single-channel output
                image_predictions.append(predicted_density)
            
            # Move to the next batch
            batch_start += batch_size
        
        # Store the predictions for the current image
        predictions.append(image_predictions)
    
    return predictions  # Return predictions for all images

### Model Evaluation and Prediction with Caffe

In [ ]:
# Define the model definition file (deploy configuration)
model_definition = os.path.join(model_path, 'deploy.prototxt')

# Loop through each set of model weights (.caffemodel) in the weights directory
for weights_file in glob.glob(os.path.join(weights_path, '*.caffemodel')):
    # Load the network with the model definition and corresponding weights
    net = caffe.Net(model_definition, weights_file, caffe.TEST)
    
    # Set the computation mode to GPU or CPU based on availability
    if HAS_GPU:
        caffe.set_device(GPU_ID)
        caffe.set_mode_gpu()
    else:
        caffe.set_mode_cpu()
    
    # Start the timer to measure the prediction time
    start_time = time.time()
    
    # Perform predictions on the test set
    predictions = predict(image_test, VGG_ILSVRC_16_layers_mean)
    
    # Print the time taken for predictions
    print('Prediction time:', time.time() - start_time)
    
    # Calculate the sum of density maps for the test set (ground truth and predictions)
    ground_truth_counts = np.array(list(map(np.sum, density_test)))  # Ground truth counts
    predicted_counts = np.array(list(map(np.sum, predictions)))  # Predicted counts

    # Calculate and print the Mean Absolute Error (MAE) for the current model weights
    mae = np.average(np.abs(predicted_counts - ground_truth_counts))
    print(f'{weights_file} MAE: {mae}')